In [1]:
import torch
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed
import random as rand
import matplotlib.pyplot as plt
import pickle 

import sys
sys.path.append('../common/')
from utils import *
from optimisation import *
from loo_cv import *

In [2]:
seed=0
torch.manual_seed(seed)

In [3]:
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [4]:
class Net(nn.Module):
    def __init__(self,N=10):
            super(Net, self).__init__()
            
            nInputs = int(N*N/2-N/2)
            
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(in_features=nInputs, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=32),
                nn.ReLU(),
                nn.Linear(in_features=32, out_features=16),
                nn.ReLU(),
                nn.Linear(in_features=16, out_features=1)
            )
            # map to positive
            
            
    def forward(self, x):
            logits = self.linear_relu_stack(x)
            return logits

# generate features

In [5]:
def upper_half(A):
    n = A.shape[0]
    return 1/A[np.triu_indices(n, k = 1)]

In [6]:
# network to be tested:
directory = "plots_bvp/"
if not os.path.exists(directory):
    os.makedirs(directory)

n=10
path =   '../network/results_sorted_1d_2d/best_model.pt'
model=Net()
model.load_state_dict(torch.load(path))
model.eval()

Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=45, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=16, bias=True)
    (9): ReLU()
    (10): Linear(in_features=16, out_features=1, bias=True)
  )
)

# boundary value problem 

In [7]:
# two dimensional linear elliptic boundary value problem, for example
# u_xx+u_yy=-4*math.pi**2*math.sin(2*math.pi*x*y)*(x**2+y**2), (x,y) \in [a,b]*[c,d]
# u(x,y)= math.sin(2*math.pi*x*y), (x,y) \in boundary
# u_exact=math.sin(2*math.pi*x*y)

In [8]:
import math
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed
from random import random
from scipy.spatial import distance
import pickle
from rbf_tools import *
from optimisation import *
from one_dim import *
from two_dim import *


r = lambda x,y: math.sqrt(x**2 + y**2)  #distance
DD = lambda f,r, x: -f**2/((1 + f**2*r**2)**(3/2)) +3* f**4/((1 + f**2*r**2)**(5/2)) * x**2  # Second derivative with respect to

def phi(f, x, y):
    z = (1 + (f * np.linalg.norm(x-y)) ** 2) ** (-0.5)
    return z



In [9]:
def euclidean_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))

def find_nearest_neighbors(points):
    num_points = points.shape[0]
    distances = np.full(num_points, np.inf)
    
    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                distance = euclidean_distance(points[i], points[j])
                if distance < distances[i]:
                    distances[i] = distance
    
    return distances
# Function to compute the Euclidean distance between two points
def dist(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

# Function to check if a point is inside a circle
def is_inside_circle(p, c):
    return dist(p, c[:2]) <= c[2]

# Function to compute the circle from 3 points
def circle_from_3_points(p1, p2, p3):
    A = p2[0] - p1[0]
    B = p2[1] - p1[1]
    C = p3[0] - p1[0]
    D = p3[1] - p1[1]
    E = A * (p1[0] + p2[0]) + B * (p1[1] + p2[1])
    F = C * (p1[0] + p3[0]) + D * (p1[1] + p3[1])
    G = 2 * (A * (p3[1] - p2[1]) - B * (p3[0] - p2[0]))
    
    if G == 0:  # Collinear points
        return None
    
    cx = (D * E - B * F) / G
    cy = (A * F - C * E) / G
    r = dist((cx, cy), p1)
    
    return (cx, cy, r)

# Function to compute the circle from 2 points
def circle_from_2_points(p1, p2):
    cx = (p1[0] + p2[0]) / 2
    cy = (p1[1] + p2[1]) / 2
    r = dist(p1, p2) / 2
    return (cx, cy, r)

# Recursive function to find the minimum enclosing circle
def welzl(P, R, n):
    if n == 0 or len(R) == 3:
        if len(R) == 0:
            return (0, 0, 0)
        elif len(R) == 1:
            return (R[0][0], R[0][1], 0)
        elif len(R) == 2:
            return circle_from_2_points(R[0], R[1])
        else:
            return circle_from_3_points(R[0], R[1], R[2])
    
    idx = rand.randint(0, n - 1)
    p = P[idx]
    P[idx], P[n - 1] = P[n - 1], P[idx]
    
    d = welzl(P, R, n - 1)
    
    if is_inside_circle(p, d):
        return d
    
    return welzl(P, R + [p], n - 1)

# Function to find the minimum enclosing circle
def find_min_circle(points):
    P = points[:]
    #random.shuffle(P)
    return welzl(P, [], len(P))

In [10]:
def get_bvp_vector(r,DD,eval_point,x_axis,eps,n):
    B=np.zeros((n+1,1)) 
    for j in range(n):
        dis= r(eval_point[0]-x_axis[j][0],eval_point[1]-x_axis[j][1])
        B[j,0]=  DD(eps,dis,eval_point[0]-x_axis[j][0])+DD(eps,dis,eval_point[1]-x_axis[j][1])
    
    return B

In [11]:
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import cdist

def two_bvp(shape_type,r,DD,model,u_exact,rhs,no_testcase,N,oversampling,a,b,c,d,n):        
    origin=np.array([0,0])       
    x=get_structured_points_modified(a,b,c,d,N)
    evall=get_structured_points_modified(a,b,c,d,oversampling*N)
    
    no_interpolation_points= x.shape[0]
    no_evaluation_points= evall.shape[0]

    true_vals=np.zeros((no_evaluation_points,1))
    for i in range(no_evaluation_points):
        true_vals[i,0]=u_exact(evall[i][0],evall[i][1])
     
    distance =cdist(evall,x)
    closest = np.argsort(distance, axis=1)   
    scaled_training_set_flat = []
    x_coords = []
    sorted_indices=[]

    eps_final_vector=np.zeros((no_evaluation_points,1))
    
    # generate features
    for i in range(no_evaluation_points):
        x_local=np.zeros((n,2))
        for j in range(n):
            x_local[j]=x[closest[i][j]]
        #Compute Euclidean distances from the origin (0, 0)
        distance = np.linalg.norm(x_local, axis=1)
        # Get the indices that would sort the array based on the distances
        sorted_index = np.argsort(distance)
        sorted_indices.append(sorted_index)
        
        x_local = sorted( x_local, key = lambda x: np.linalg.norm(x - origin ) )
        x_local = np.reshape(x_local, (n,2)) 
        x_coords.append(x_local)        
        x_axis=x_local.reshape(n,2)

        if shape_type=='hardy':
            nearest_distances = find_nearest_neighbors(x_axis)
            d=np.sum(nearest_distances)/n        
            eps_final_vector[i]=1/(0.815*d)
        elif shape_type=='franke':
            x_axis_list=x_axis.tolist()
            circle = find_min_circle(x_axis_list)
            eps_final_vector[i]=0.8*(10**(1/2))/(2 * circle[2])
        elif shape_type=='mfranke':
            x_axis_list=x_axis.tolist()
            circle = find_min_circle(x_axis_list)
            eps_final_vector[i]=0.8*(10**(1/4))/(2 * circle[2])  
        
        xxx=generate_distance_from_coordinates(x_axis)              
        training_set_distances_flatten = upper_half(xxx)
        scaled_training_set_flat.append(training_set_distances_flatten)
    if shape_type=='NN':
        scaled_training_set_tensor = torch.tensor(scaled_training_set_flat,dtype=torch.float)
        scaled_training_set_tensor.reshape((no_evaluation_points,int(n*n/2-n/2)))
    
        model.eval()
        eps_final= model(scaled_training_set_tensor)
        eps_final_vector = eps_final.detach().numpy()
    
    D=np.zeros((no_evaluation_points,no_evaluation_points))
    rhs_vals=np.zeros((no_evaluation_points,1))
    for i in range(no_evaluation_points):
        eps = eps_final_vector[i][0]
        L =  get_int_matrix(x_coords[i],eps)        
        Lpoisson=get_bvp_vector(r,DD,evall[i],x_coords[i],eps,n)
            
        w=np.linalg.solve(L,Lpoisson)       
        
        D[i,closest[i][:n][sorted_indices[i]]]=w[:n].reshape(n,)
        
        rhs_vals[i]=rhs(evall[i][0],evall[i][1])
    
    non_boundary_indices=[]
    boundary_indices=[]
    for i in range(no_evaluation_points):
        if (not np.isclose(evall[i][0],a)) and (not np.isclose(evall[i][0],b)) and (not np.isclose(evall[i][1],c)) and (not np.isclose(evall[i][1],d)):
            non_boundary_indices.append(i)
        else:
            boundary_indices.append(i)
            
    boundary_condition=np.matmul(D[np.ix_(non_boundary_indices,boundary_indices)],true_vals[boundary_indices])

    approximator_oversampled=np.zeros((no_evaluation_points,1))
    approximator_oversampled[non_boundary_indices]=np.linalg.solve(D[np.ix_(non_boundary_indices,non_boundary_indices)],rhs_vals[non_boundary_indices]-boundary_condition)
    approximator_oversampled[boundary_indices]=true_vals[boundary_indices]
    
    error=0
    for j in range(no_evaluation_points):
        error+=(true_vals[j]-approximator_oversampled[j])**2.0
    final_tot_error = math.sqrt(error/no_evaluation_points)
    return final_tot_error


In [12]:
oversampling=1
a,c=0.0,0.0
b,d=1.0,1.0   

In [ ]:
n=10 # length of stencil
RHS1 =lambda x,y: -4*math.pi**2*math.sin(2*math.pi*x*y)*(x**2+y**2)
U1 =lambda x,y: math.sin(2*math.pi*x*y)

no_testcase=1
nx=[20,40,60,80,100,120]

for rhs,u_exact in zip([RHS1],[U1]):
    no_test=no_testcase
    x_vec=[]
    error=dict()
    for N in nx:
        x_vec.append(N*N)
        for shape_type in ['hardy','franke','mfranke','NN']:
            final_error=two_bvp(shape_type,r,DD,model,u_exact,rhs,no_testcase,N,oversampling,a,b,c,d,n)
            if shape_type in error.keys():
                error[shape_type].append(final_error)
            else:
                error[shape_type]=[final_error]
             
            print(final_error) 
    fig=plt.figure()
    plt.loglog(x_vec,error['hardy'],color='orange',marker='o')
    plt.loglog(x_vec,error['franke'],color='g',marker='o')
    plt.loglog(x_vec,error['mfranke'],color='purple',marker='o')
    plt.loglog(x_vec,error['NN'],color='r',marker='o')
    plt.tick_params(axis='both', which='major', labelsize=13, width=2, length=5)

    plt.xlabel('M', fontsize=13)
    plt.ylabel('Error', fontsize=13)
    plt.legend(["Hardy","Franke","Modified Franke","NN"], fontsize=13,loc ="best")       

    plt.savefig(directory+'testcase'+str(no_testcase)+'.png',bbox_inches='tight', dpi=150) 
    plt.close()
    
    no_testcase+=1  